# Metaprogramming

One of the key mantras in programming is to not repeat yourself. Whenever you do
the same task more than once, you should take a look at it to see if there is any way to
automate it. This is the primary reason for writing programs. But this applies equally to
the task of writing the code itself. If you are repeating chunks of code, you should step
back to see if there is some better way of achieving the same result.
One technique available to deal with this issue is metaprogramming. Essentially,
metaprogramming is code that affects other code. The usual way this is done within
Python is either with decorators, metaclasses, or descriptors.
### 8-1. Using a Function Decorator to Wrap Existing Code

You want to alter the behavior of an already existing function by wrapping it with other
code. This wrapper code can then be swapped in or out from different modules if they use
the same decorator name, allowing you to alter the original function in different ways.
<br>
A function can be wrapped by adding a decorator to the top of the function definition,
using a label that begins with an ampersand.
<br>
Listing 8-1 shows an example where you use the decorators provided by the
line_profile module.

#### Listing 8-1.  Using the Profile Decorator

In [ ]:
from line_profile import *
@profile
def my_adder(x, y):
    return x + y

This code wraps the function my_adder() with profiling code from the line_profile
module. This module is not part of the standard library, so you need to install it onto your
system. This is the same as explicitly wrapping one function with another, as in Listing 8-2.
### Listing 8-2.  Wrapping a Function

In [ ]:
from line_profiler import *
def my_adder(x, y):
    return x + y

my_adder = profile(my_adder)

### 8-2. Writing a Function Decorator to Wrap Existing Code
Problem
You want to write a wrapper for a function to add extra functionality.
<br>
Python includes the wraps keyword that defines a function that can wrap another
function and be used as a decorator.
<br>
In order to write a decorator of your own, you need to use the wraps keyword from
the functools module. This keyword is used as a decorator to help define your own
new decorator. Listing 8-3 shows an example that prints out the function name of the
decorated function.
### Listing 8-3.  A Decorator to Print Out Function Names

In [ ]:
from functools import wraps
def function_name(func):
    message = func.__qualname__
    @wraps(func)
    def wrapper((*args, **kwargs)):
        print(message)
        return func(*args, **kwargs)
    return wrapper

You can then use it just like any other decorator, as in Listing 8-4.
### Listing 8-4.  Using Your Own Decorator

In [ ]:
@function_name
def adder(x,y):
    return x+y

###  8-3. Unwrapping a Decorated Function

You need to get access to the functionality of a function that has been decorated.
<br>
You can get the original unwrapped function by using the __wrapped__ attribute of the
function.
<br>
Assuming that the decorator was correctly coded using the wraps function from
functools, then you can get the original function by using the __wrapped__ attribute, as
in Listing 8-5.
### Listing 8-5.  Getting the Unwrapped Function

In [ ]:
adder(2,3)
adder.__wrapper__(2,3)

###  8-4. Using a Metaclass to Change the Construction of a Class
You need to add extra functionality to a class, similar to a decorator for functions. This
can be done by changing what object a class is an instance of by using a metaclass.
<br>
Metaclasses can be used in a similar fashion to using subclasses.
<br>
When using a metaclass, you include it in the class definition, as in Listing 8-6.
### Listing 8-6.  Using a Metaclass

In [ ]:
class my_counter(metaclass=Singleton):
    pass

By default, a class is an instance of the type class. The code in Listing 8-6 makes the
new class an instance of the Singleton class rather than the type class. You can also set
the metaclass within the class definition, as in Listing 8-7.
#### Listing 8-7.  Setting the __metaclass__ Attribute

In [ ]:
class my_counter():
    __metaclass__ = Singleton
    pass

In both Listing 8-6 and 8-7, your new class is created as an instance of the Singleton
class. This is one method of using the singleton design pattern within Python.
###  8-5. Writing a Metaclass

You need to change how a class is instantiated by writing your own metaclass.
<br>
With the use of a metaclass, you can redefine how a class is actually instantiated, allowing
you to create classes that can only be instantiated once (the singleton design pattern), or are
cached, for example. These examples are used for logging classes or streaming data parsers.
<br>
You create a metaclass by building a class that overwrites one or more of the functions
used during instantiation. For example, you could override the __call__ function to
create a class that cannot be instantiated, as in Listing 8-8.
### Listing 8-8.  A Metaclass That Stops Instantiation

In [ ]:
class CannotInit(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Cannot instantiate")

Now, when you try to use it as a metaclass and directly instantiate the new class, you
will get an exception raised.
If you need more complicated behavior, as in a singleton, for example, you can
override multiple functions, as in Listing 8-9.
### Listing 8-9.  Creating a Singleton Metaclass

In [ ]:
class MySingleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
def __call__(self, *args, **kwargs):
    if self.__instance is None:
        self.__instance = super().__call__(*args, **kwargs)
        return self.__instance
    else:
        return self.__instance

This code traps the both the instantiation and the calling of any class that uses this
metaclass so that only one instance can exist at a single time.
###  8-6. Using Signatures to Change the Parameters a Function Accepts
You want to be able to control the signature for a function at runtime. This allows you to
dynamically change what parameters a function accepts. For example, you can force your
function to only use keyword parameters in one case, and then allow for positional or
keyword parameters in another case.
<br>
The inspect module includes the tools needed to create and use signatures for functions.


The example in Listing 8-10 shows how to create a new signature.
### Listing 8-10.  Creating a Signature

In [ ]:
from inspect import Signature, Parameter
params = [Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),Parameter('z', Parameter.KEYWORD_ONLY, default=None)]
my_sig = Signature(params)
print(my_sig)
(x, y=42, *, z=None)

In this code, you use the Parameter class to create a list of function parameters. There
are keywords for each type of parameter. One thing to note is that if you have a list of
keyword-only parameters in a normal function definition, you use an asterisk to mark which
parameters are keyword only. This shows up when you print out the newly created signature.
To use this signature, you can use the bind method to take the generic lists of
positional and keyword parameters and bind them to the parameters within the signature
you created. An example is given in Listing 8-11.
### Listing 8-11.  Using a Signature

In [ ]:
def my_func(*args, **kwargs):
    bound_values = my_sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name, value)

This way, you can have the same function bind the parameters using different
signatures, based on how you need to process them.